- In this notebook, we will add the collected data from google map and wikipedia to our merged dataset.
- also we will clean the data and prepare it for database insertion.

In [1]:
import numpy as np
import pandas as pd

**we have 5 fildes to work with:**
1. merged dataset: merged_datasets/france_monuments.csv
2. base dataset codinates: monument_urls_with_coords.csv wich should be add to merged dataset
3. google map data: paris_monuments_ratings.csv (from this we want to fill some missing data in merged dataset + adding some new columns)
    - columns: [name, rating, review_count, website,opening_hours, google_maps_url]
4. wikipedia data: paris_monuments_wiki.csv (from this we want to fill some missing data in merged dataset + adding some new columns)
    - columns: [input_name, wiki_name, wiki_description, wiki_url, category]
5. redirect_log.txt: this file is showing potential irelevant data in the wikipedia data file that should be checked cause we didn't found a proper enlish or french match and we selected first related page from wikipedia search.
    - in each line we have: input_name -> redirected_url

# Add base coordinates to merged dataset

In [3]:
df_merged = pd.read_csv("merged_datasets/france_monuments.csv")
print(df_merged.shape)
df_merged.head(2)

(564, 15)


,name,url,short_description,ticket_price,ticket_price_conditions,opening_hours,payment_methods,address,visiting_services,ticket_price_raw,city,category,lat,lng,Tourpedia_id
0,Basilique Cathédrale de Saint-Denis,https://www.saint-denis-basilique.fr/,Discover the necropolis of the kings and queen...,17.0,Free for under 26s,From October to March\nMonday to Saturday: 10:...,coins | bank cards | cheques | Culture cheques...,"1, rue de la Légion d'Honneur, 93200 Saint-Denis",Cabin baggage ('cabines') allowed: maximum 40x...,Price: 17 €,NaN,NaN,NaN,NaN,NaN
1,Oppidum et musée archéologique d'Ensérune,https://www.enserune.fr/,Discover a major Gallo-Roman city located betw...,9.0,Free for those under 26 years old.,Please note: last access to the monument one h...,cash | bank cards | cheques | Culture cheques ...,"2901 route d’Ensérune, 34440 Nissan-lez-Ensérune",Drinking fountain | Toilets | Wheelchair | Pos...,Price: 9 €,NaN,NaN,NaN,NaN,NaN


In [4]:
df_base_coords = pd.read_csv("monument_urls_with_coords.csv")
print(df_base_coords.shape)
df_base_coords.head(2)

(94, 4)


,name,url,latitude,longitude
0,Abbaye de Beaulieu-en-Rouergue,https://www.beaulieu-en-rouergue.fr/,44.2101,1.8540
1,Abbaye de Charroux,https://www.abbaye-charroux.fr/,46.1437,0.4046


In [5]:
df_base = pd.read_csv("paris_monuments_translated.csv")
print(df_base.shape)
df_base.head(2)

(88, 10)


,name,url,short_description,ticket_price,ticket_price_conditions,opening_hours,payment_methods,address,visiting_services,ticket_price_raw
0,Basilique Cathédrale de Saint-Denis,https://www.saint-denis-basilique.fr/,Discover the necropolis of the kings and queen...,17.0,Free for under 26s,From October to March\nMonday to Saturday: 10:...,coins | bank cards | cheques | Culture cheques...,"1, rue de la Légion d'Honneur, 93200 Saint-Denis",Cabin baggage ('cabines') allowed: maximum 40x...,Price: 17 €
1,Oppidum et musée archéologique d'Ensérune,https://www.enserune.fr/,Discover a major Gallo-Roman city located betw...,9.0,Free for those under 26 years old.,Please note: last access to the monument one h...,cash | bank cards | cheques | Culture cheques ...,"2901 route d’Ensérune, 34440 Nissan-lez-Ensérune",Drinking fountain | Toilets | Wheelchair | Pos...,Price: 9 €


In [6]:
# check if urls in base coords are the same as in base so we can merge based on url

len(df_base_coords.url.unique()), len(df_base.url.unique()), len(set(df_base_coords.url.unique()) - set(df_base.url.unique()))

(94, 88, 6)

In [7]:
df_merged.url.value_counts().sort_values(ascending=False)

url
http://palais-royal.monuments-nationaux.fr    2
https://www.enserune.fr/                      1
https://www.saint-denis-basilique.fr/         1
https://www.tresor-cathedrale-autun.fr/       1
https://www.chateau-angers.fr/                1
                                             ..
http://www.photoquai.fr                       1
http://www.ndsaintsacrement.org               1
http://mam.paris.fr                           1
http://www.frederikskirkenparis.dk            1
http://www.tour-eiffel.fr                     1
Name: count, Length: 119, dtype: int64

In base cords and base dataset we have exactly one unique url for each monument and based dataset urls are subset of base coords dataset urls.\
But As you can see, in the merged dataset, there is one url that that repeated 2 times and rest are unique.\
So, if that one is not in the base coords dataset, we will have no problem mergin based on that.

In [8]:
# check if the repeated urls in merged dataset not exist in the base coords dataset urls
(
    set(
        df_base_coords.url.unique()
    ) - set(
        df_merged.url.value_counts()[df_merged.url.value_counts() > 1].index.tolist()
    )
) >= set(df_base_coords.url.unique())

True

Based on the previous check we can safly join the base coords and with merged dataset on "url" column.

In [9]:
df_merged_with_cords = df_merged.copy()

coords = (
    df_base_coords
    .set_index('url')[['latitude', 'longitude']]
    .rename(columns={'latitude': 'lat', 'longitude': 'lng'})
)

# # Only assign for URLs that exist in both dataframes
urls = pd.Index(df_base['url'])

df_merged_with_cords['prev_index'] = df_merged_with_cords.index
df_merged_with_cords = df_merged_with_cords.set_index('url', drop=False)
df_merged_with_cords.loc[urls, ['lat', 'lng']] = coords.reindex(urls)[['lat', 'lng']]
df_merged_with_cords = df_merged_with_cords.set_index('prev_index', drop=True)
df_merged_with_cords.index.name = None

print('nan values count in "lat" and "lng" columns:', df_merged_with_cords.lat.isna().sum(), df_merged_with_cords.lng.isna().sum())
df_merged_with_cords.head(2)

nan values count in "lat" and "lng" columns: 0 0


,name,url,short_description,ticket_price,ticket_price_conditions,opening_hours,payment_methods,address,visiting_services,ticket_price_raw,city,category,lat,lng,Tourpedia_id
0,Basilique Cathédrale de Saint-Denis,https://www.saint-denis-basilique.fr/,Discover the necropolis of the kings and queen...,17.0,Free for under 26s,From October to March\nMonday to Saturday: 10:...,coins | bank cards | cheques | Culture cheques...,"1, rue de la Légion d'Honneur, 93200 Saint-Denis",Cabin baggage ('cabines') allowed: maximum 40x...,Price: 17 €,NaN,NaN,48.935461,2.359835,NaN
1,Oppidum et musée archéologique d'Ensérune,https://www.enserune.fr/,Discover a major Gallo-Roman city located betw...,9.0,Free for those under 26 years old.,Please note: last access to the monument one h...,cash | bank cards | cheques | Culture cheques ...,"2901 route d’Ensérune, 34440 Nissan-lez-Ensérune",Drinking fountain | Toilets | Wheelchair | Pos...,Price: 9 €,NaN,NaN,43.310300,3.115200,NaN


In [10]:
# df_merged_with_cords.to_csv("merged_datasets/with_coords/france_monuments.csv", index=False)

# Add Google Maps data to merged dataset
- from this dataset we want to add the following columns to the merged dataset:
    - google_map_rating
    - google_map_review_count
    - website (if missing in merged dataset)
    - opening_hours (if missing in merged dataset)
    - google_maps_url

In [11]:
df_google_maps = pd.read_csv("paris_monuments_ratings.csv")
print(df_google_maps.shape)
df_google_maps.head(2)

(564, 6)


,name,rating,review_count,website,opening_hours,google_maps_url
0,Basilique Cathédrale de Saint-Denis,4.6,7419.0,https://www.saint-denis-basilique.fr/,Thursday=10 am–4:45 pm; Friday=10 am–4:45 pm; ...,https://www.google.com/maps/place/Basilique+Ca...
1,Oppidum et musée archéologique d'Ensérune,4.3,666.0,NaN,NaN,https://www.google.com/maps/search/Oppidum%20e...


check possible inconsistencies in google maps data before merging using 

In [12]:
# check current urls are okey
import re

print("Nans in url column:", df_merged_with_cords.url.isna().sum())
df_merged_with_cords.url.apply(lambda x: bool(re.match(r'^https?://', x)) if pd.notna(x) else True).all()

Nans in url column: 444


np.True_

check if what percetage of current urls matched with google maps dataset urls to check quality of urls in google maps dataset.

In [13]:
def get_domain(url):
    if pd.isna(url): return None
    match = re.search(r'https?://(?:www\.)?([^/]+)', str(url))
    return match.group(1) if match else None

(
    df_merged_with_cords.url.loc[
        ~df_merged_with_cords.url.isna() & ~df_google_maps.website.isna()
    ].apply(get_domain) == df_google_maps.website.loc[
        ~df_merged_with_cords.url.isna() & ~df_google_maps.website.isna()
    ].apply(get_domain)
).mean()

np.float64(0.7830188679245284)

In [14]:
import math

def distance_m(lat1, lon1, lat2, lon2):
    """Great-circle distance (Haversine) in meters between two lat/lon points."""
    R = 6371000.0  # Earth radius in meters

    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)

    a = math.sin(dphi / 2)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(dlambda / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

In [15]:
def extract_coords_from_url(url):
    """Extract latitude and longitude from a Google Maps URL."""
    if pd.isna(url):
        return None
    match = re.search(r'@([-+]?\d*\.\d+),([-+]?\d*\.\d+)', url)
    if match:
        return float(match.group(1)), float(match.group(2))
    return None

google_map_cords = df_google_maps.google_maps_url.loc[
    df_google_maps.google_maps_url.str.startswith('https://www.google.com/maps/place')
].apply(extract_coords_from_url)
(
    df_merged_with_cords.loc[google_map_cords.index, ['lat', 'lng']].apply(
        lambda row: distance_m(
            row['lat'],
            row['lng'],
            google_map_cords.loc[row.name][0],
            google_map_cords.loc[row.name][1]
        ),
        axis=1
    ) < 1000
).mean()


np.float64(0.4657534246575342)

# We suddenly stop here
as the google maps data has low quality (the locations are far from the actual monuments), we decided to stop using it and recollect it by official google maps api with proper location filtering.